<a href="https://colab.research.google.com/github/anguoyuan/Diffusion-for-retrievl-python/blob/main/diffusion_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a tutorial about diffusion for image retrieval instead of generation. It is mainly based on Iscen's CVPR17 [paper](https://openaccess.thecvf.com/content_cvpr_2017/papers/Iscen_Efficient_Diffusion_on_CVPR_2017_paper.pdf) and the[ code](https://github.com/ducha-aiki/manifold-diffusion) of Dmytro Mishkin. Beginners might find Fan Yang's [slides](https://github.com/fyang93/diffusion/blob/master/slides.pdf) useful for understanding diffusion (but its implementation sometimes is not as effective as the following standard one). Please use the following code if you prefer high accuracy.  

## Key Insights on Diffusion

* Diffusion, a widely recognized reranking technique in retrieval, has proven its efficacy across numerous datasets and features.

* Diffusion can enhance retrieval performance without additional training.

* A notable variant of diffusion is PageRank, a technique that played a key role in Google's significant business success.

* Although traditionally reranking might increase time costs, recent studies have successfully transitioned the process to an offline mode, thereby significantly minimizing online costs. For further understanding, refer to the [NeurIPS Hypergraph](https://github.com/anguoyuan/Hypergraph-Propagation-and-Community-Selection-for-Objects-Retrieval) and [AAAI offline diffusion](https://github.com/fyang93/diffusion/tree/master) studies.









In [16]:
import os
import numpy as np
from scipy.io import loadmat
from scipy.sparse import csr_matrix, eye, diags
from scipy.sparse import linalg as s_linalg
from time import time

Firstly, prepare some useful functions.

* sim_kernel: A commonly used similarity kernel in the retrieval field. For more details, refer to the "Regional Diffusion" paper by Iscen, in Section 5.1.

* normalize_connection_graph: This function computes the symmetric normalization of the affinity matrix, aiding in data structure interpretation.

* topK_W: This function change a full affinity graph into a k-NN graph. Since the original affinity matrix can become unmanageable due to its quadratic relation to the number of vectors in the database, this approach is necessary. Truncation to a k-NN structure maintains nearly the same level of accuracy.







In [2]:
def sim_kernel(dot_product):
    return np.maximum(np.power(dot_product,3),0)

def normalize_connection_graph(G):
    W = csr_matrix(G)
    W = W - diags(W.diagonal()) # sets the diagonal entries of the matrix W to zero. zero self-similarities
    D = np.array(1./ np.sqrt(W.sum(axis = 1))) # degree matrix of the graph. It is the normalization factor, the content is inverse of the squre root of the sum of each row of W,
    D[np.isnan(D)] = 0
    D[np.isinf(D)] = 0
    D_mh = diags(D.reshape(-1))
    Wn = D_mh * W * D_mh #normalization
    return Wn

def topK_W(G, K = 100):
    sortidxs = np.argsort(-G, axis = 1)
    for i in range(G.shape[0]):
        G[i,sortidxs[i,K:]] = 0
    G = np.minimum(G, G.T)
    return G

Prepare the initial search result.

In [4]:
X=np.random.rand(2048,4996)
Q=np.random.rand(2048,70)

In [5]:
K = 100 # approx 50 mutual nns
QUERYKNN = 10
R = 2000
alpha = 0.9

In [6]:
sim  = np.dot(X.T, Q)
qsim = sim_kernel(sim).T
sortidxs = np.argsort(-qsim, axis = 1)
for i in range(len(qsim)):
    qsim[i,sortidxs[i,QUERYKNN:]] = 0

Find the normalized laplacian matrix

In [7]:
qsim = sim_kernel(qsim)
A = np.dot(X.T, X)
W = sim_kernel(A).T
W = topK_W(W, K)
Wn = normalize_connection_graph(W)

<ipython-input-2-fc2c72ad4698>:7: RuntimeWarning: divide by zero encountered in true_divide
  D = np.array(1./ np.sqrt(W.sum(axis = 1))) # degree matrix of the graph. It is the normalization factor, the content is inverse of the squre root of the sum of each row of W,


Try the diffusion. We use MINimum RESidual iteration to solve *(I-αWn)f=b*, where *(I-αWn)* is the Laplacian of the normalized affinity matrix, *b* is the initial ranking result for each query, and the solved *f* is the result of diffusion.  

In [8]:
def cg_diffusion(qsims, Wn, alpha = 0.99, maxiter = 10, tol = 1e-3):
    Wnn = eye(Wn.shape[0]) - alpha * Wn # Laplacian matrix
    out_sims = []
    for i in range(qsims.shape[0]):
        #f,inf = s_linalg.cg(Wnn, qsims[i,:], tol=tol, maxiter=maxiter)
        f,inf = s_linalg.minres(Wnn, qsims[i,:], tol=tol, maxiter=maxiter) #There are several famous solvers for the linear system. Here, it uses MINimum RESidual iteration instead of Conjugate Gradient
        out_sims.append(f.reshape(-1,1))
    out_sims = np.concatenate(out_sims, axis = 1)
    ranks = np.argsort(-out_sims, axis = 0)
    return ranks


In [9]:
cg_ranks =  cg_diffusion(qsim, Wn, alpha)

In [11]:
cg_ranks.shape

(4996, 70)

In cases where the database is excessively large, an effective approach is to truncate the affinity matrix. This is achieved by retaining only the rows and columns pertaining to the top-ranked image regions and subsequently re-normalizing the matrix.

It's important to note that this process is distinct from topK_W, which is responsible for the construction of the k-NN graph.







In [12]:
def find_trunc_graph(qs, W, levels = 3):
    needed_idxs = []
    needed_idxs = list(np.nonzero(qs > 0)[0])
    for l in range(levels):
        idid = W.nonzero()[1]
        needed_idxs.extend(list(idid))
        needed_idxs =list(set(needed_idxs))
    return np.array(needed_idxs), W[needed_idxs,:][:,needed_idxs]

def dfs_trunk(sim, A,alpha = 0.99, QUERYKNN = 10, maxiter = 8, K = 100, tol = 1e-3):
    qsim = sim_kernel(sim).T
    sortidxs = np.argsort(-qsim, axis = 1)
    for i in range(len(qsim)):
        qsim[i,sortidxs[i,QUERYKNN:]] = 0
    qsims = sim_kernel(qsim)
    W = sim_kernel(A)
    W = csr_matrix(topK_W(W, K))
    out_ranks = []
    t =time()
    for i in range(qsims.shape[0]):
        qs =  qsims[i,:]
        tt = time()
        w_idxs, W_trunk = find_trunc_graph(qs, W, 2);
        Wn = normalize_connection_graph(W_trunk)
        Wnn = eye(Wn.shape[0]) - alpha * Wn
        f,inf = s_linalg.minres(Wnn, qs[w_idxs], tol=tol, maxiter=maxiter)
        ranks = w_idxs[np.argsort(-f.reshape(-1))]
        missing = np.setdiff1d(np.arange(A.shape[1]), ranks)
        out_ranks.append(np.concatenate([ranks.reshape(-1,1), missing.reshape(-1,1)], axis = 0))
    #print time() -t, 'qtime'
    out_ranks = np.concatenate(out_ranks, axis = 1)
    return out_ranks


In [17]:
cg_trunk_ranks =  dfs_trunk(sim, A, alpha = alpha, QUERYKNN = QUERYKNN )

<ipython-input-2-fc2c72ad4698>:7: RuntimeWarning: divide by zero encountered in true_divide
  D = np.array(1./ np.sqrt(W.sum(axis = 1))) # degree matrix of the graph. It is the normalization factor, the content is inverse of the squre root of the sum of each row of W,
